## Necessary Libraries and Constants

In [ ]:
#https://www.youtube.com/watch?v=F1HA7e3acSI
#https://www.bustabit.com/play
#https://bitcointalk.org/index.php?topic=2807542.0
#https://jsfiddle.net/Dexon95/2fmuxLza/

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import hashlib
import random
import string
import hmac
import math
import codecs

# https://www.blockchain.com/btc/block/0000000000000000004d6ec16dafe9d8370958664c1dc422f452892264c59526
salt505750 = "0000000000000000004d6ec16dafe9d8370958664c1dc422f452892264c59526"
SALT = salt505750

## Get Result and Get Previous Game Function

In [ ]:
results = [('319aa8124bbcddac2bae8438cfd5e658aeca56d524736739622dfb0a09942b22', 159.93),
('c64559640b124fa354eaaee31510eaa0f1a5fdd5aa004363b95693e52d274d28',2.36),
('c263a30ad8820e8c3d46f9a5db3c34d3b081cd349a11367a9fbd52063f17cd34',1.31),
('a6bb64e83b8efa52524ba526cd036ae17c3ec00b9fbec49b8cd4dc1321d7b4ce',1.09),
('17612b37d97f0faf0930762a328ab383bd2e107c5cc5c975cc76405f41a4bbc2',1.43)]


def get_prev_game(hash_code):
    m = hashlib.sha256()
    m.update(hash_code.encode("utf-8"))
    return m.hexdigest()
for i in range(len(results)-1):
    assert get_prev_game(results[i][0]) == results[i+1][0]
print("get_prev_game passed")

def get_result(seed, salt=SALT):
    salt = salt.encode("utf8")
    seed = codecs.decode(bytes(seed, "utf8"), "hex")  #https://tinyurl.com/2bf4dann
    hm = hmac.new(salt, seed, hashlib.sha256)
    h = hm.hexdigest()
    
    r = int(h[:13], 16)       # 52 most significant bits as int
    x = r / 2**52             # [0,1)
    x = 99 / (1-x)            # [99.0, 4.458563631096791e+17]
    result = math.floor(x)    # [99, 445856363109679104]
    return (max(1, result/100))
    
for res in results[:]:   
    assert getResult(res[0]) == res[1], print(f"\n\n{res}: {getResult(res[0])}")
print("get_result passed")

In [ ]:
def sim(numerator=99):
    results = []
    for x in range(0,1000):
        x = x/1000                           # [0,0.999]
        x = numerator/(1-x)                  # [99, 99/(0.001)  99,000]
        result = math.floor(x)               # [99, 99000]
        answer = max(1, result/100)          # [1, 990]
        results.append(answer)
    return results
        
game = sim()
fair = sim(100)

print("game", min(game), sum(game)/len(game), max(game))
print("fair?", min(fair), sum(fair)/len(fair), max(fair))

if False:
    for g in game:
        print(g)

## Collecting All Game Results

In [ ]:
game_hash = '4fc2f264ea3e0bbf8405f74f0bf354bef946283d6023352f5f8d441d6aaad861' # 29oct2021 Update to latest game's hash for more results
first_game = "86728f5fc3bd99db94d3cdaf105d67788194e9701bf95d049ad0e1ee3d004277"  #https://bitcointalk.org/index.php?topic=2807542.0

results = []
count = 0
while game_hash != first_game:
    count += 1
    results.append(get_result(game_hash))
    game_hash = get_prev_game(game_hash)
    
results = np.array(results)

In [ ]:
print(count)

## Testing Probability Formula

In [ ]:
# What's the analytical EV of this new algorithm?  

## Testing Expected Value Formula

In [ ]:
#realized results from start of game to now   
#confirms a 1% house edge
multiplier = 1.7
(results < multiplier).mean() * -1 + (multiplier - 1)*(results >= multiplier).mean()

## Visualizations

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})

plt.hist(results, range=(0, 25))
plt.title("Histogram of Game Results", fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel("Number of Games", fontsize=15)
plt.xlabel("Multiplier", fontsize=15)

In [ ]:
def calculate_ev(multiplier):
    # This is for the version 1 algorithm, not the version 2.
    return ((1/33) + (32/33)*(.01 + .99*(1 - 1/(multiplier-.01))))*-1 + (multiplier-1)*(1 - ((1/33) + (32/33)*(.01 + .99*(1 - 1/(multiplier-.01)))))

xs = np.linspace(101, 1001, 901) / 100
ys = [calculate_ev(x) for x in xs]
y2s = [(results < x).mean() * -1 + (x - 1)*(results >= x).mean() for x in xs]

plt.plot(xs, ys, linewidth=5)
plt.xlabel("Multiplier", fontsize=15)
plt.ylabel("Expected Value", fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylim(-.045, -.000)
plt.plot(xs, y2s, linewidth=3)
plt.title("Expected Value by Multiplier", fontsize=20)
plt.legend(["(error) Theoretical Results", "Actual Results"])

## Martingale Test

In [ ]:
negatives = []
in_a_row = 0
for multiplier in results:
    if multiplier < 2:
        in_a_row += 1
    else:
        in_a_row = 0
    negatives.append(in_a_row)
negatives = np.array(negatives)

In [ ]:
for i in range(1, 14):
    print("Probability of Losing %d Game(s) in a Row:"%i, (negatives >= i).mean())